In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

import ROOT
import root_numpy as rn

matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='Times New Roman'

from ROOT import sp
sp.LoadCombined()

In [ ]:
df=pd.DataFrame(rn.root2array(["../bin/filteredoutput_osc_mc_detail_1.root",
                               "../bin/filteredoutput_osc_mc_detail_2.root"],
                               selection='Weight>0',treename='MiniBooNE_CCQE'))

In [ ]:
#Set Eqe and BkgdID via sp
df['Eqe']    = df.apply(lambda x : sp.CombinedFit_EnuQE_ryan(x['Energy'],x['CosTheta'],3),axis=1)
df['BkgdID'] = df.apply(lambda x : sp.CombinedFit_bkgd_type(x['NUANCEChan'],x['NuType']),axis=1)

In [ ]:
#
# Profile view of True Enu
#
xlo=200
xhi=2000
dx =100
bins=np.arange(xlo,xhi+dx,dx)
pass_df = df.query("(NuType==3 | NuType==4) & NUANCEChan==1")

fig,ax=plt.subplots(figsize=(10,6))

data1   = pass_df.NuMomT.values*1000.0
weight1 = pass_df.Weight.values

thist = ax.hist(data1,
                weights=weight1,
                bins=bins,
                histtype='stepfilled',
                label=['Nu_e CCQE'])#,'All Signal'])

ax.legend(loc='best')
ax.set_xlabel("Enu",fontweight='bold')
ax.set_xlim(300,2000)
ax.grid()
plt.show()


x = thist[0]
print x
print data1.size

In [ ]:
#
# Profile view of reconstructed Eqe
#
pass_df = df.query("PassOsc==1 & (NuType==3 | NuType==4) & NUANCEChan==1 & NuMomT > @bins[0]/1000.0 & NuMomT < @bins[-1]/1000.0")


fig,ax=plt.subplots(figsize=(10,6))

data2   = pass_df.Eqe.values*1000.0
weight2 = pass_df.Weight.values

thist = ax.hist(data2,
                weights=weight2,
                bins=bins,
                histtype='stepfilled',
                label='Selected')

ax.legend(loc='best')
ax.set_xlabel("Eqe",fontweight='bold')
ax.set_xlim(300,2000)
ax.grid()

plt.show()
print data1.size
b = thist[0]
print b

In [ ]:
#
# Make migration matrix
#

#choose model here
the_df = df.query("PassOsc==1 & (NuType==3 | NuType==4) & NUANCEChan==1")

fig,ax=plt.subplots(figsize=(10,6))

data3   = the_df.NuMomT.values*1000.0
weight3 = the_df.Weight.values

data4   = the_df.Eqe.values*1000.0
weight4 = the_df.Weight.values

thist = ax.hist2d(data3,data4,weights=weight3,bins=[bins,bins])
                  
ax.legend(loc='best')
ax.set_xlabel("Enu",fontweight='bold')
ax.set_ylabel("Eqe",fontweight='bold')
ax.grid()
plt.colorbar(thist[3])
plt.show()
A=thist[0]
A_ = A.T / x 

In [ ]:
np.set_printoptions(3)
#
# Recover original reco spectrum
#
b_ = np.dot(A_,x)
fig,ax=plt.subplots(figsize=(10,6))
ax.hist(bins[:-1],bins=bins,weights=b ,histtype='step',linestyle='dashed',color='black')
ax.hist(bins[:-1],bins=bins,weights=b_,histtype='step',color='blue')
ax.set_xlim(300,2000)
ax.grid()
ax.set_xlabel("Folded",fontweight='bold')
plt.show()

print "b"
print b
print
print "b_"
print b_
print
print "b-b_"
print b-b_

In [ ]:
#
# SVD
#

# numpy SVD
U, s, V = np.linalg.svd(A_, full_matrices=True)

#
# Calculate with true reco
#

d = np.dot(U.T,b)
z = d / s
z = np.nan_to_num(z)
x_ = np.dot(V.T,z)

print "s"
print s
print
print "z"
print z
print
print "d"
print d
print
print "x_"
print x_
print
print "V[-2]"
print V[-2]
print

In [ ]:
plt.plot(s,'-o')
ax=plt.gca()
ax.set_xlabel("$s_i$",fontweight='bold')
ax.set_yscale('log')
plt.show()

plt.plot(np.abs(d),'-o')
ax=plt.gca()
ax.grid()
ax.set_yscale('log')
ax.hlines(1,0,len(d))
ax.set_xlabel("$|d_i|$",fontweight='bold')
plt.show()

In [ ]:
#
# Recover MC distribution
#

fig,ax=plt.subplots(figsize=(10,6))
ax.hist(bins[:-1],bins=bins,weights=x ,histtype='step',lw=2,color='black',linestyle='dashed')

x__ = np.where((x_<1e9) & (x_>-1e9),x_,0.0)
ax.hist(bins[:-1],bins=bins,weights=x__,histtype='step',lw=2,color='blue')
ax.set_xlim(300,2000)
ax.grid()
ax.set_xlabel("Folded",fontweight='bold')
plt.show()

In [ ]:
#
# Poisson fluctuation
#
b__ = np.random.poisson(b_)
fig,ax=plt.subplots(figsize=(10,6))
ax.hist(bins[:-1],bins=bins,weights=b_ ,histtype='step',lw=2,color='black',linestyle='dashed')
ax.hist(bins[:-1],bins=bins,weights=b__,histtype='step',color='green',lw=2)
ax.set_xlim(300,2000)
ax.grid()
ax.set_xlabel("Folded",fontweight='bold')
plt.show()

In [ ]:
# normalize columns
A_ = A.T / x 

# numpy SVD
U, s, V = np.linalg.svd(A_)

#
# Calculate with poisson fluctuated
#
print "s"
print s
print
d = np.dot(U.T,b__)
z = np.nan_to_num(d / s)
x_ = np.dot(V.T,z)
print "z"
print z
print
print "d"
print d
print
print "x_"
print x_

In [ ]:
#
# Recover MC with poisson fluctuated reco
#
fig,ax=plt.subplots(figsize=(10,6))
ax.hist(bins[:-1],bins=bins,weights=x ,histtype='step',lw=2,color='black',linestyle='dashed')

x__ = np.where((x_<1e9) & (x_>-1e9),x_,0.0)
ax.hist(bins[:-1],bins=bins,weights=x__,histtype='step',lw=2)
ax.set_xlim(300,2000)
ax.grid()
ax.set_xlabel("Folded",fontweight='bold')
plt.show()

In [ ]:
#
# Try out the regulator
#

#Build C
sz = bins.size-1
#sz=5
a0 = np.ones(sz)
a1 = np.ones(sz-1)

C = -2.0*np.diag(a0,0) + np.diag(a1,-1) + np.diag(a1,1)
C[0,0] = -1
C[-1,-1] = -1

epsi = np.power(10.0,-4)
C += np.identity(sz) * epsi
C_inv = np.linalg.inv(C)

In [ ]:
A_C = np.dot(A_,C_inv)

# numpy SVD
U, s, V = np.linalg.svd(A_C)

d = np.dot(U.T,b__)
k=5
tau = s[k] * s[k]
z = (d * s) / (s * s + tau)
x_ = np.dot(C_inv,np.dot(V.T,z))

In [ ]:
plt.plot(s,'-o')
ax=plt.gca()
ax.set_yscale('log')
ax.grid()
ax.set_xlabel("$s_i$")
plt.show()

plt.plot(np.abs(d),'-o')
ax=plt.gca()
ax.set_yscale('log')
ax.grid()
ax.set_xlabel("$d_i$")
plt.show()


In [ ]:
#
# Recover MC with poisson fluctuated reco
#
fig,ax=plt.subplots(figsize=(10,6))
ax.hist(bins[:-1],bins=bins,weights=x ,histtype='step',lw=2,color='black',linestyle='dashed')

ax.hist(bins[:-1],bins=bins,weights=x_,histtype='step',lw=2)
ax.set_xlim(300,2000)
ax.grid()
ax.set_xlabel("Folded",fontweight='bold')
plt.show()

In [ ]:
tsvd = sp.TikhonovSVD()

In [ ]:
A__ = A_.astype(np.float32)
eigen_A = sp.as_mat_float32(A__.copy())

In [ ]:
tsvd.Initialize(eigen_A)

In [ ]:
epsi

In [ ]:
A__

In [ ]:
for i in xrange(10):
    print tsvd.A()(0,i),eigen_A(0,i),A__[0,i]

In [ ]:
sp.as_array_float32(tsvd.C()).diagonal()

In [ ]:
plt.imshow(sp.as_array_float32(tsvd.A_C_inv()) -  np.dot(A__.astype(np.float32),np.linalg.inv(C.astype(np.float32))),
           interpolation='none')

ax=plt.gca()
plt.colorbar()
plt.show()


plt.imshow(sp.as_array_float32(sp.Invert(tsvd.C())) -  np.linalg.inv(C.astype(np.float32)),
           interpolation='none')

ax=plt.gca()
plt.colorbar()
plt.show()

In [ ]:
np.linalg.inv(C.astype(np.float32))[:,-1]

In [ ]:
sp.as_array_float32(sp.Invert(tsvd.C()))[:,-1]

In [ ]:
print list(b__)
k = sp.as_vector_float32(b__.astype(np.float32))
print [i for i in b__]

In [ ]:
k_eigen = sp.to_vector_eigen(k)

In [ ]:
res=tsvd.Unfold(k_eigen,5)

In [ ]:
sp.as_array_float32(res)

In [ ]:
#
# Recover MC with poisson fluctuated reco
#
fig,ax=plt.subplots(figsize=(10,6))
ax.hist(bins[:-1],bins=bins,weights=x ,histtype='step',lw=2,color='black',linestyle='dashed')

_x = sp.as_array_float32(res)
ax.hist(bins[:-1],bins=bins,weights=_x,histtype='step',lw=2)
ax.set_xlim(300,2000)
ax.grid()
ax.set_xlabel("Folded",fontweight='bold')
plt.show()

In [ ]:
plt.plot(sp.as_array_float32(tsvd.s()),'-o')
ax=plt.gca()
ax.set_yscale('log')
ax.grid()
plt.show()

plt.plot(np.abs(sp.as_array_float32(tsvd.d())),'-o')
ax=plt.gca()
ax.set_yscale('log')
ax.grid()
plt.show()

In [ ]:
sp.as_array_float32(tsvd.d()) * sp.as_array_float32(tsvd.s())